<a href="https://colab.research.google.com/github/shahin1009/EDMD/blob/main/EDMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extended Dynamic Mode Decomposition (EDMD)
###(basically for finding koopman eignfuncions)

## Introduction:
This notebook presents an in-depth analysis of nonlinear dynamical systems, focusing on performing data-driven analysis. The notebook covers different stages of analysis, starting from the numerical integration of ordinary differential equations (ODEs) to the implementation of the Extended Dynamic Mode Decomposition (EDMD) method.

Suppose you have dynamic measurements of a system, **specifically y and dy**. First you need to build a dictinary of observables using the data

## Building the dictionary of observables



In [ ]:
import numpy as np

def buildTheta(yin, nVars, polyorder):


    """
    Builds a polynomial dictionary from input data.

    Args:
        yin: Input data.
        nVars: Number of input variables.
        polyorder: Polynomial order.

    Returns:
        Polynomial dictionary.
    """

    n = yin.shape[0]
    ind = 0
    yout = np.zeros((n, nVars))

    # Copy input variables to output
    for i in np.arange(nVars):
        yout[:, ind] = yin[:, i]
        ind += 1

    if polyorder >= 2:
        # Polynomial order 2
        for i in np.arange(nVars):
            for j in np.arange(i, nVars):
                new_col = yin[:, i] * yin[:, j]
                yout = np.append(yout, new_col.reshape(-1, 1), 1)
                ind += 1

    if polyorder >= 3:
        # Polynomial order 3
        for i in np.arange(nVars):
            for j in np.arange(i, nVars):
                for k in np.arange(j, nVars):
                    new_col = yin[:, i] * yin[:, j] * yin[:, k]
                    yout = np.append(yout, new_col.reshape(-1, 1), 1)
                    ind += 1

    if polyorder >= 4:
        # Polynomial order 4
        for i in np.arange(nVars):
            for j in np.arange(i, nVars):
                for k in np.arange(j, nVars):
                    for l in np.arange(k, nVars):
                        new_col = yin[:, i] * yin[:, j] * yin[:, k] * yin[:, l]
                        yout = np.append(yout, new_col.reshape(-1, 1), 1)
                        ind += 1


    return yout


In [ ]:
def buildGamma(yin, ydotin, nVars, polyorder):
    n = yin.shape[0]
    ind = 0
    yout = np.zeros((n, nVars))
    for i in np.arange(nVars):
        yout[:, ind] = ydotin[:, i]
        ind += 1

    if polyorder >= 2:
        # poly order 2
        for i in np.arange(nVars):
            for j in np.arange(i, nVars):
                new_col = ydotin[:, i] * yin[:, j] + yin[:, i] * ydotin[:, j]
                yout = np.append(yout, new_col.reshape(-1, 1), 1)
                ind += 1

    if polyorder >= 3:
        # poly order 3
        for i in np.arange(nVars):
            for j in np.arange(i, nVars):
                for k in np.arange(j, nVars):
                    new_col = (
                        ydotin[:, i] * yin[:, j] * yin[:, k]
                        + yin[:, i] * ydotin[:, j] * yin[:, k]
                        + yin[:, i] * yin[:, j] * ydotin[:, k]
                    )
                    yout = np.append(yout, new_col.reshape(-1, 1), 1)
                    ind += 1

    if polyorder >= 4:
        # poly order 4
        for i in np.arange(nVars):
            for j in np.arange(i, nVars):
                for k in np.arange(j, nVars):
                    for l in np.arange(k, nVars):
                        new_col = (
                            ydotin[:, i] * yin[:, j] * yin[:, k] * yin[:, l]
                            + yin[:, i] * ydotin[:, j] * yin[:, k] * yin[:, l]
                            + yin[:, i] * yin[:, j] * ydotin[:, k] * yin[:, l]
                            + yin[:, i] * yin[:, j] * yin[:, k] * ydotin[:, l]
                        )
                        yout = np.append(yout, new_col.reshape(-1, 1), 1)
                        ind += 1

    return yout


## For printing purposes:

In [ ]:
import numpy as np

def poolDataLIST(yin, ahat, nVars, polyorder):
    n = len(yin)+1
    ind = 0
    yout = []

    for i in range(nVars):
        yout.append([yin[i]])
        ind += 1

    if polyorder >= 2:
        for i in range(nVars):
            for j in range(i, nVars):
                yout.append([yin[i], yin[j]])
                ind += 1

    if polyorder >= 3:
        for i in range(nVars):
            for j in range(i, nVars):
                for k in range(j, nVars):
                    yout.append([yin[i], yin[j], yin[k]])
                    ind += 1

    if polyorder >= 4:
        for i in range(nVars):
            for j in range(i, nVars):
                for k in range(j, nVars):
                    for l in range(k, nVars):
                        yout.append([yin[i], yin[j], yin[k], yin[l]])
                        ind += 1

    if polyorder >= 5:
        for i in range(nVars):
            for j in range(i, nVars):
                for k in range(j, nVars):
                    for l in range(k, nVars):
                        for m in range(l, nVars):
                            yout.append([yin[i], yin[j], yin[k], yin[l], yin[m]])
                            ind += 1

    output = yout

    for item1, item2 in zip(output, ahat):
      print(f'{item1}\t{item2}'.replace(", ", "*"))
    return output


## Generating the data from a known dynamic

In [ ]:
from scipy.integrate import odeint
from scipy.linalg import svd

def f(q, t):
    x, y = q
    return [y, x-x**3]

def solve(ic,t):
    sol = odeint(f, ic, t)
    return sol

t = np.linspace(0, 10, 10000)
x0 = np.array([0, -2.8])
y=solve(x0,t)
dy = np.array(f(y.T,t)).T

## Extended Dynamic Mode Decomposition (EDMD) using Singular Value Decomposition (SVD):

1- Specify the desired polynomial order (polyorder) and number of variables (nvar) for the EDMD.

2- Build the library matrix Theta using the data y, number of variables, and polynomial order. The library matrix captures the polynomial basis functions of the data.

3- Build the library matrix Gamma using the data y and its derivative dy, along with the number of variables and polynomial order. The library matrix Gamma captures the polynomial basis functions of the time derivatives.

4- Perform Singular Value Decomposition (SVD) on the difference between a zero matrix and the library matrix Theta multiplied by a scalar Gamma. This step helps identify the dominant modes in the data.

5- Extract the last row of the right singular vectors matrix V (corresponding to the smallest singular value) and assign it to the sparse vector xi0. The sparse vector represents the coefficients of the dominant modes.

6- Set small values in xi0 to zero.

7- Generate the output using the sparse vector xi0 by calling poolDataLIST() with the appropriate arguments. This step constructs the output using the identified sparse coefficients.

In [ ]:
# Construct libraries
polyorder = 4
nvar = 2

# Build the library matrix Theta
Theta = buildTheta(y, nvar, polyorder)

# Build the library matrix Gamma
Gamma = buildGamma(y, dy, nvar, polyorder)

# Perform SVD
U, S, V = svd(0 * Theta - Gamma, full_matrices=False)

# Extract the sparse vector xi0
xi0 = V[-1, :]
xi0[np.abs(xi0) < 1e-3] = 0

# Generate the output using the sparse vector
output = poolDataLIST(['x', 'y', 'z'], xi0, nvar, polyorder)


['x']	0.0
['y']	0.0
['x'*'x']	-0.6666666666666667
['x'*'y']	0.0
['y'*'y']	0.6666666666666669
['x'*'x'*'x']	0.0
['x'*'x'*'y']	0.0
['x'*'y'*'y']	0.0
['y'*'y'*'y']	0.0
['x'*'x'*'x'*'x']	0.33333333333333337
['x'*'x'*'x'*'y']	0.0
['x'*'x'*'y'*'y']	0.0
['x'*'y'*'y'*'y']	0.0
['y'*'y'*'y'*'y']	0.0


## Implementation of the least-squares Koopman algorithm breakdown:

1-Specify the desired polynomial order (polyorder) and number of variables (nvar).

2-Build the library matrix Theta using the data y and the specified number of variables and polynomial order.

3-Build the library matrix Gamma using the data y and its derivative dy, along with the specified number of variables and polynomial order.

4-Compute the Koopman operator K using the pseudo-inverse of Theta multiplied by Gamma. **This step is the core of the least-squares approach.**

5-Set small values in K to zero to enhance numerical stability.

6-Compute the eigenvalues (D) and eigenvectors (T) of K using np.linalg.eig().

7-Sort the eigenvalues (D) in ascending order and obtain the sorted indices (IX) using np.argsort().

8-Extract the eigenvector corresponding to the smallest eigenvalue from the eigenvector matrix T and assign it to xi1.


9-Generate the output using the absolute value of xi1 by calling poolDataLIST() to get the polynomial coefficients.

In [ ]:
# Define the desired polynomial order and number of variables
polyorder = 4
nvar = 2

# Build the library matrix Theta from data (y)
Theta = buildTheta(y, nvar, polyorder)

# Build the library matrix Gamma from data (y and dy)
Gamma = buildGamma(y, dy, nvar, polyorder)

# Compute the Koopman operator using the least-squares approach
K = np.linalg.pinv(Theta) @ Gamma

# Set small values in K to zero for numerical stability
K[np.abs(K) < 1e-12] = 0

# Compute the eigenvalues (D) and eigenvectors (T) of K
D, T = np.linalg.eig(K)

# Sort the eigenvalues (D) in ascending order and obtain the sorted indices (IX)
IX = np.argsort(np.abs(D))

# Extract the eigenvector (xi1) corresponding to the smallest eigenvalue
xi1 = T[:, IX[0]]

# Set small values in xi1 to zero for numerical stability
xi1[np.abs(xi1) < 1e-12] = 0

# Generate the output using the absolute value of xi1
output = poolDataLIST(['x', 'y'], abs(xi1), nvar, polyorder)


['x']	0.0
['y']	0.0
['x'*'x']	0.6666666666666792
['x'*'y']	0.0
['y'*'y']	0.6666666666666536
['x'*'x'*'x']	0.0
['x'*'x'*'y']	0.0
['x'*'y'*'y']	0.0
['y'*'y'*'y']	0.0
['x'*'x'*'x'*'x']	0.33333333333333426
['x'*'x'*'x'*'y']	0.0
['x'*'x'*'y'*'y']	0.0
['x'*'y'*'y'*'y']	0.0
['y'*'y'*'y'*'y']	0.0


## Same thing for Lorenz:

For better results, we need better dataset for lorenz system. The accuracy and effectiveness of EDMD for the Lorenz system, or any other system, depend on several factors such as **the quality and quantity of the data**, the choice of basis functions, the selection of the polynomial order, and the presence of noise or modeling errors.

In [ ]:
# Lorenz Derivitive
def lorenz_deriv(x_y_z, t0, sigma=10, beta=8/3, rho=28):
    # Unpack the input variables
    x, y, z = x_y_z

    # Compute the derivatives
    dx_dt = sigma * (y - x)
    dy_dt = x * (rho - z) - y
    dz_dt = x * y - beta * z

    # Return the derivatives
    return [dx_dt, dy_dt, dz_dt]


def solve_lorenz(ic, t, sigma=10, beta=8/3, rho=28):
    # Solve the Lorenz system of equations using odeint
    sol = odeint(lorenz_deriv, ic, t, args=(sigma, beta, rho))
    return sol

# Define the time steps for integration
t = np.linspace(0, 100, 100000)

# Set the initial conditions
x0 = [0, 1, 0]

# Solve the Lorenz system
y = solve_lorenz(x0, t)

# Compute the derivative of the solution
dy = np.array(lorenz_deriv(y.T, t)).T


# Construct libraries
polyorder = 3
nvar = 3

# Build the library matrix Theta
Theta = buildTheta(y, nvar, polyorder)

# Build the library matrix Gamma
Gamma = buildGamma(y, dy, nvar, polyorder)

# Perform SVD
U, S, V = svd(0 * Theta - Gamma, full_matrices=False)

# Extract the sparse vector xi0
xi0 = V[-1, :]
xi0[np.abs(xi0) < 1e-3] = 0

# Generate the output using the sparse vector
output = poolDataLIST(['x', 'y', 'z'], xi0, nvar, polyorder)


['x']	0.8847513519784561
['y']	-0.46259824963530666
['z']	0.00486311460869955
['x'*'x']	0.0030044790835703126
['x'*'y']	-0.002256090478753538
['x'*'z']	-0.05109122509834033
['y'*'y']	0.0
['y'*'z']	0.023772354342215706
['z'*'z']	0.0
['x'*'x'*'x']	-0.0018422852901978162
['x'*'x'*'y']	0.0
['x'*'x'*'z']	0.0
['x'*'y'*'y']	0.0
['x'*'y'*'z']	0.0
['x'*'z'*'z']	0.0
['y'*'y'*'y']	0.0
['y'*'y'*'z']	0.0
['y'*'z'*'z']	0.0
['z'*'z'*'z']	0.0


In [ ]:
# Define the desired polynomial order and number of variables
polyorder = 3
nvar = 3

# Build the library matrix Theta
Theta = buildTheta(y, nvar, polyorder)

# Build the library matrix Gamma
Gamma = buildGamma(y, dy, nvar, polyorder)

# Compute the Koopman operator using the least-squares approach
K = np.linalg.pinv(Theta) @ Gamma

# Set small values in K to zero for numerical stability
K[np.abs(K) < 1e-12] = 0

# Compute the eigenvalues (D) and eigenvectors (T) of K
D, T = np.linalg.eig(K)

# Sort the eigenvalues (D) in ascending order and obtain the sorted indices (IX)
IX = np.argsort(np.abs(D))

# Extract the eigenvector (xi1) corresponding to the smallest eigenvalue
xi1 = T[:, IX[0]]

# Set small values in xi1 to zero for numerical stability
xi1[np.abs(xi1) < 1e-3] = 0

# Generate the output using the absolute value of xi1
output = poolDataLIST(['x', 'y','z'], abs(xi1), nvar, polyorder)


['x']	0.7102392779405408
['y']	0.3288441532101477
['z']	0.6006537792722473
['x'*'x']	0.1340208262509187
['x'*'y']	0.08371064498731991
['x'*'z']	0.028091434477879127
['y'*'y']	0.010307492497910598
['y'*'z']	0.009445655512111
['z'*'z']	0.02544555878173035
['x'*'x'*'x']	0.0015105077486257405
['x'*'x'*'y']	0.003140279951581945
['x'*'x'*'z']	0.0034600304249564674
['x'*'y'*'y']	0.0018072957924786949
['x'*'y'*'z']	0.0026980185262491315
['x'*'z'*'z']	0.0
['y'*'y'*'y']	0.0
['y'*'y'*'z']	0.0
['y'*'z'*'z']	0.0
['z'*'z'*'z']	0.0
